In [ ]:
from ISG2TIF.zutilsISG import isg2tif_f1, isg2tif_f2
from glob import glob
import os

software matlab to help conversion to geotif 
= https://www.isgeoid.polimi.it/Software/software.html

In [8]:
isg_fpath = r"c:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\igs\Brazil_2015_MAPGEO2015_gravG_20201222.isg"
tif_fpath = isg_fpath.replace('.isg','_A.tif')
#isg2tif(input_file=isg_fpath,output_file=tif_fpath)

In [13]:
idpath = r"c:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\igs"
odpath = r"c:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\tif"

idpath = r"C:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\DEPLOY\ISG"
odpath = r"C:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\DEPLOY\TIF"
os.makedirs(odpath, exist_ok=True)
print(f"{len(os.listdir(idpath))} num of files")
os.listdir(idpath)

4 num of files


['Africa_2019_AFRGEOID2019_gravG_20210625.isg.txt',
 'AGP2007_20160415.isg.txt',
 'ALASKA_GEOID96_20160414.isg.txt',
 'USGG2003a_20161106.isg.txt']

In [14]:
failed_list = []
end1 = '.isg'
end2 = '.isg.txt'
ifiles = glob(f"{idpath}\\*")
for fi in ifiles:
   # print(fi)
    bn = os.path.basename(fi)
    if bn.endswith(end1):
        bn2 = bn.replace(end1, '.tif')
    elif bn.endswith(end2):
        bn2 = bn.replace(end2, '.tif')
    fo = os.path.join(odpath, bn2)
    #print(fo)
    try:
        isg2tif_f2(input_file=fi,output_file=fo)
    except:
        try:
            isg2tif_f1(isg_fpath=fi,output_file=fo)
        except:
            
            print('Failled!')
            print(fi)
            failed_list.append(fi)
        


(301, 293)
Failled!
C:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\DEPLOY\ISG\Africa_2019_AFRGEOID2019_gravG_20210625.isg.txt
(877, 829)
isg2tif_f1 write to C:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\DEPLOY\TIF\AGP2007_20160415.tif
(691, 931)
isg2tif_f1 write to C:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\DEPLOY\TIF\ALASKA_GEOID96_20160414.tif
Failled!
C:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\DEPLOY\ISG\USGG2003a_20161106.isg.txt


In [15]:
# get data from Diangti, and keep improving the code to account for other cases too 
# study the code from matlab

In [16]:
failed_list

['C:\\Users\\Joseph\\Downloads\\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\\DEPLOY\\ISG\\Africa_2019_AFRGEOID2019_gravG_20210625.isg.txt',
 'C:\\Users\\Joseph\\Downloads\\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\\DEPLOY\\ISG\\USGG2003a_20161106.isg.txt']

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

# Step 1: Parse the metadata
metadata = {
    'ISG format': '1.01',
    'delta lat': '0.083333330000',
    'delta lon': '0.083333330000',
    'lat max': '24.041666665324',
    'lat min': '7.958333331676',
    'lon max': '110.041666665314',
    'lon min': '101.958333331686',
    'model name': 'GEOID_FFT',
    'model type': 'gravimetric',
    'ncols': '97',
    'nodata': '-9999.0000',
    'nrows': '193',
    'reference': 'WGS84',
    'units': 'meters'
}

# Step 2: Extract key parameters
delta_lat = float(metadata['delta lat'])  # Resolution in latitude (decimal degrees)
delta_lon = float(metadata['delta lon'])  # Resolution in longitude (decimal degrees)
lat_min = float(metadata['lat min'])      # Minimum latitude (decimal degrees)
lat_max = float(metadata['lat max'])      # Maximum latitude (decimal degrees)
lon_min = float(metadata['lon min'])      # Minimum longitude (decimal degrees)
lon_max = float(metadata['lon max'])      # Maximum longitude (decimal degrees)
nrows = int(metadata['nrows'])            # Number of rows
ncols = int(metadata['ncols'])            # Number of columns
nodata = float(metadata['nodata'])        # No-data value
crs = "EPSG:4326"                         # CRS for WGS84

# Step 3: Define the affine transformation
transform = rasterio.transform.from_origin(
    west=lon_min,
    north=lat_max,
    xsize=delta_lon,
    ysize=delta_lat
)

# Step 4: Prepare the data array (example array with random values)
data_array = np.random.rand(nrows, ncols)  # Replace with your actual data array

# Step 5: Write the raster file
output_file = "output_raster_v2.tif"
with rasterio.open(
    output_file,
    'w',
    driver='GTiff',
    height=nrows,
    width=ncols,
    count=1,  # Single band
    dtype=data_array.dtype,
    crs=crs,
    transform=transform,
    nodata=nodata
) as dst:
    dst.write(data_array, 1)

print(f"Raster file saved as {output_file}")